In [1]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
import requests
from models import *
from scrape import *

In [2]:
engine = create_engine('sqlite:///basketball.db')
Session = sessionmaker(bind = engine)
session = Session()

In [3]:
def Instantiate(year1,year2):
    coachsite_objects = CoachSite()
    lowered = list(map(lambda school : school.lower(),coachsite_objects.salaries[1]))
    school_list = list(map(lambda school : school.replace(' ', '-'), lowered))
    teamsite_objects = {}
    for school in school_list:
        teamsite_objects[school] = TeamSite(school,year1,year2)
    return teamsite_objects, coachsite_objects

In [4]:
everything = (Instantiate(2011,2018))
teamsite = everything[0] # gives you a dictionary with keys = school name and values\n",
             # are TeamSite instances\n",
keys = list((teamsite).keys())
coachsite = everything[1] #gives you object instance\n",

#######Instansiate coach model##########
coach_instances = []
for tup in coachsite.salaries[0]:
    tup2 = int(tup[2].replace('$','').replace(',',''))
    c_instance = Coach(name = tup[0] , years = tup[1] , salary = tup2,\
                       sal_school = tup[3])

    session.add(c_instance)
    coach_instances.append(c_instance)

#######Instansiate school model##########
team_instances = []
for school_name in keys:
    years_ = teamsite[school_name].years
    winloss_ = teamsite[school_name].winloss
    team_points_ = teamsite[school_name].team_points
    opp_points_ = teamsite[school_name].opp_points

    for i in range(0,len(years_)):
        if years_[i] != '' and winloss_[i] != '' and team_points_[i] != '' and opp_points_[i] != '':
            team_instance = School(name = school_name, years = years_[i], winloss = winloss_[i],\
                            team_points = team_points_[i], opp_points = opp_points_[i])
            for coach in coach_instances:
                if coach.sal_school.replace(' ', '-').lower() == team_instance.name and coach.years == (team_instance.years)[:-3]:
                    team_instance.coaches.append(coach)
                    session.add(team_instance)
                    team_instances.append(team_instance)                   

In [5]:
session.commit()

In [16]:
vars(team_instance.coaches[0])

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1231a2550>,
 'name': 'Chris Beard',
 'years': '2016',
 'salary': 260000,
 'sal_school': 'Arkansas-Little Rock',
 'schools': [<models.School at 0x12343de80>]}

In [11]:
team_instance.coaches

[]